# 04 — Infer Videos → PGN (Submission)

Use board warp + per‑cell CNN + rule engine to output PGN per video.

- Reads videos from `data/public/videos/*.mp4` (local) or a Kaggle input path you specify.
- Loads model from `models/cell_cnn.h5`.
- Writes `submissions/submission.csv` with `row_id,output`.


In [ ]:
# %%capture
# !pip install --quiet opencv-python python-chess tqdm


In [5]:
# # --- imports & env ---
# import os, sys, csv
# from pathlib import Path
# import json
# import cv2
# import numpy as np
# from collections import deque
# from tqdm import tqdm
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# # --- env / root selection (local vs Kaggle) ---
# ON_KAGGLE = Path('/kaggle').exists()
# ROOT = Path('/kaggle/working') if ON_KAGGLE else Path('..')

# # ให้ import แพ็กเกจจากโฟลเดอร์ src ได้ (เหมือนโน้ตบุ๊กก่อนหน้า)
# sys.path.insert(0, str(ROOT / 'src'))

# # --- project modules ---
# from Chess_Detection_Competition.utils import load_config
# from Chess_Detection_Competition.board import warp_board, split_grid
# from Chess_Detection_Competition.model import load_model as load_cell_model
# from Chess_Detection_Competition.pgn import diff_to_move, san_list_to_pgn, labels_to_board

# # --- paths ---
# VIDEOS_DIR = ROOT / 'data/public/videos'          # เปลี่ยนตรงนี้ได้ถ้าใช้ input ของ Kaggle competition
# MODEL_PATH = ROOT / 'models/cell_cnn.h5'
# CLASSES_JSON = ROOT / 'models/classes.json'
# OUT_DIR    = ROOT / 'submissions'
# OUT_DIR.mkdir(parents=True, exist_ok=True)
# SUBMIT_CSV = OUT_DIR / 'submission.csv'

# # --- load config / params ---
# cfg = load_config()  # อ่าน configs/parameters.yaml
# BOARD_CFG = cfg["board"]
# IMG_SIZE  = cfg["cells"]["img_size"]
# SMOOTH_K  = cfg["inference"]["smooth_k"]
# SAMPLE_STEP = cfg["inference"]["sample_step"]

# # --- load class order (ต้องตรงกับตอนเทรน) ---
# if CLASSES_JSON.exists():
#     with open(CLASSES_JSON, "r", encoding="utf-8") as f:
#         CLASSES = json.load(f)
# else:
#     # ถ้าไม่มีไฟล์นี้ ให้ใช้ลำดับคลาสเดียวกับที่เทรนประกาศไว้ในโปรเจกต์
#     CLASSES = ['BB','BK','BN','BP','BQ','BR','Empty','WB','WK','WN','WP','WQ','WR']

# print("Videos :", VIDEOS_DIR.resolve())
# print("Model  :", MODEL_PATH.resolve())
# print("Submit :", SUBMIT_CSV.resolve())
# print("Classes:", CLASSES)

# =========================
# 04 — Setup (robust & auto)
# =========================

# --- stdlib / third-party ---
import os, sys, csv, json
from pathlib import Path
from collections import deque
import cv2
import numpy as np
from tqdm import tqdm
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# --- env: local vs Kaggle ---
ON_KAGGLE = Path("/kaggle").exists()
ROOT = Path("/kaggle/working") if ON_KAGGLE else Path("..").resolve()

# ให้ import จากโฟลเดอร์ src และแพ็กเกจในโปรเจกต์ได้ (กันพลาดทั้งสองแบบ)
sys.path.insert(0, str(ROOT / "src"))
sys.path.insert(0, str(ROOT))

# --- project modules (with safe import) ---
try:
    from Chess_Detection_Competition.utils import load_config
    from Chess_Detection_Competition.board import warp_board, split_grid
    from Chess_Detection_Competition.model import load_model as load_cell_model
    from Chess_Detection_Competition.pgn import diff_to_move, san_list_to_pgn, labels_to_board
except Exception as e:
    # เผื่อบางเครื่องยังไม่ได้ pip install -e .
    # ลอง import แบบโมดูลใน src โดยตรง
    from Chess_Detection_Competition.utils import load_config  # จะสำเร็จเพราะ sys.path ใส่ ROOT/src ไว้แล้ว
    from Chess_Detection_Competition.board import warp_board, split_grid
    from Chess_Detection_Competition.model import load_model as load_cell_model
    from Chess_Detection_Competition.pgn import diff_to_move, san_list_to_pgn, labels_to_board

# --- paths ---
VIDEOS_DIR   = ROOT / "data/public/videos"     # วิดีโอ input
MODEL_PATH   = ROOT / "models/cell_cnn.h5"     # ไฟล์โมเดลที่เทรน
CLASSES_JSON = ROOT / "models/classes.json"    # เก็บลำดับคลาสตอนเทรน
TRAIN_DIR    = ROOT / "data/final/train"       # ใช้เดาจากโฟลเดอร์ (fallback)
OUT_DIR      = ROOT / "submissions"
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUBMIT_CSV   = OUT_DIR / "submission.csv"

# --- load config (fallback ให้ค่าเริ่มต้นถ้าไม่มีไฟล์) ---
try:
    cfg = load_config()  # configs/parameters.yaml
    BOARD_CFG   = cfg["board"]
    IMG_SIZE    = int(cfg["cells"]["img_size"])
    SMOOTH_K    = int(cfg["inference"]["smooth_k"])
    SAMPLE_STEP = int(cfg["inference"]["sample_step"])
    cfg_source  = "configs/parameters.yaml"
except Exception as e:
    # ค่า default ที่ใช้ได้จริง
    BOARD_CFG = {
        "warp_size": 800,
        "canny_low": 60,
        "canny_high": 180,
        "hough_threshold": 120,
        "min_line_length": 120,
        "max_line_gap": 10,
    }
    IMG_SIZE, SMOOTH_K, SAMPLE_STEP = 96, 5, 3
    cfg_source = f"[fallback defaults] ({e})"

# --- resolve class order (ต้องตรงกับตอนเทรน) ---
def infer_classes_from_train_dir(train_dir: Path):
    """เดาลำดับคลาสจากชื่อโฟลเดอร์ย่อยใน train/ ตาม sorting ของ Keras"""
    if not train_dir.exists():
        return None
    subdirs = [p.name for p in train_dir.iterdir() if p.is_dir()]
    if not subdirs:
        return None
    return sorted(subdirs)  # image_dataset_from_directory ใช้ลำดับตาม sort แบบนี้

DEFAULT_CLASSES = ['BB','BK','BN','BP','BQ','BR','Empty','WB','WK','WN','WP','WQ','WR']

CLASSES = None
if CLASSES_JSON.exists():
    try:
        CLASSES = json.loads(CLASSES_JSON.read_text(encoding="utf-8"))
        if not isinstance(CLASSES, list) or not all(isinstance(x, str) for x in CLASSES):
            raise ValueError("classes.json format invalid")
    except Exception as e:
        CLASSES = None  # ไป fallback ต่อ

if CLASSES is None:
    # พยายามเดาจากโฟลเดอร์ train/ ถ้ามี
    guessed = infer_classes_from_train_dir(TRAIN_DIR)
    if guessed and set(guessed) == set(DEFAULT_CLASSES) and len(guessed) == len(DEFAULT_CLASSES):
        CLASSES = guessed
    else:
        CLASSES = DEFAULT_CLASSES  # สุดท้ายใช้ default ที่เราคอนโทรลได้

# บันทึกลำดับคลาสล็อกไว้ (เพื่อให้ inference ครั้งต่อไปเหมือนตอนนี้เป๊ะ)
try:
    CLASSES_JSON.parent.mkdir(parents=True, exist_ok=True)
    CLASSES_JSON.write_text(json.dumps(CLASSES, ensure_ascii=False, indent=2), encoding="utf-8")
except Exception:
    pass  # ถ้าเขียนไม่ได้ไม่เป็นไร

# --- sanity checks / summary ---
video_files = sorted(VIDEOS_DIR.glob("*.mp4"))
print("=== Environment ===")
print("Kaggle     :", ON_KAGGLE)
print("ROOT       :", ROOT)
print("Config     :", cfg_source)
print("Videos dir :", VIDEOS_DIR)
print("Model path :", MODEL_PATH)
print("Submit CSV :", SUBMIT_CSV)
print("Train dir  :", TRAIN_DIR if TRAIN_DIR.exists() else "[not found]")
print("IMG_SIZE   :", IMG_SIZE)
print("SMOOTH_K   :", SMOOTH_K)
print("SAMPLE_STEP:", SAMPLE_STEP)
print("Classes    :", CLASSES)
print("Videos found:", len(video_files))

# คำเตือนที่มีประโยชน์
if not MODEL_PATH.exists():
    print("[warn] MODEL_PATH not found ->", MODEL_PATH)
if not video_files:
    print("[warn] No .mp4 files in:", VIDEOS_DIR)
if "Empty" not in CLASSES:
    print("[warn] 'Empty' class is missing in CLASSES — results will be wrong for empty squares.")
if len(CLASSES) != 13:
    print(f"[warn] CLASSES has {len(CLASSES)} items (expected 13). Check your class order.")



=== Environment ===
Kaggle     : False
ROOT       : C:\Users\worap\Downloads\image_processing_term_orject\Chess_Detection_Competition
Config     : configs/parameters.yaml
Videos dir : C:\Users\worap\Downloads\image_processing_term_orject\Chess_Detection_Competition\data\public\videos
Model path : C:\Users\worap\Downloads\image_processing_term_orject\Chess_Detection_Competition\models\cell_cnn.h5
Submit CSV : C:\Users\worap\Downloads\image_processing_term_orject\Chess_Detection_Competition\submissions\submission.csv
Train dir  : C:\Users\worap\Downloads\image_processing_term_orject\Chess_Detection_Competition\data\final\train
IMG_SIZE   : 96
SMOOTH_K   : 5
SAMPLE_STEP: 3
Classes    : ['BB', 'BK', 'BN', 'BP', 'BQ', 'BR', 'Empty', 'WB', 'WK', 'WN', 'WP', 'WQ', 'WR']
Videos found: 5


In [7]:
# --- load trained cell-CNN ---
model = load_cell_model(str(MODEL_PATH))

# ทำนาย 8×8 ด้วย smoothing buffer ต่อ cell
def predict_labels8x8(warped_bgr, buffers):
    cells = split_grid(warped_bgr, IMG_SIZE)
    X = []
    for _, patch in cells:
        rgb = cv2.cvtColor(patch, cv2.COLOR_BGR2RGB).astype(np.float32)
        X.append(preprocess_input(rgb))
    X = np.asarray(X, dtype=np.float32)  # shape: (64, H, W, 3)

    probs = model.predict(X, verbose=0)  # (64, num_classes)

    mat = [[None]*8 for _ in range(8)]
    k = 0
    for r in range(8):
        for c in range(8):
            buffers[r][c].append(probs[k])
            avg = np.mean(np.stack(buffers[r][c], axis=0), axis=0)
            mat[r][c] = CLASSES[int(np.argmax(avg))]
            k += 1
    return mat


In [8]:
import chess

def decode_video_to_pgn(video_path: Path) -> str:
    cap = cv2.VideoCapture(str(video_path))
    ok, frame = cap.read()
    if not ok:
        cap.release()
        return ""

    # warp กระดานเฟรมแรก
    warped, _ = warp_board(frame, {"board": BOARD_CFG})

    # เตรียม buffer สำหรับ smoothing ต่อ cell
    buffers = [[deque(maxlen=SMOOTH_K) for _ in range(8)] for __ in range(8)]
    prev_labels = predict_labels8x8(warped, buffers)

    sans = []
    frame_id = 0

    while True:
        ok, frame = cap.read()
        if not ok:
            break
        frame_id += 1

        # sample ทุก N เฟรม
        if frame_id % SAMPLE_STEP:
            continue

        warped, _ = warp_board(frame, {"board": BOARD_CFG})
        now_labels = predict_labels8x8(warped, buffers)

        mv = diff_to_move(prev_labels, now_labels)
        if mv is not None:
            # ใช้ labels_to_board เพื่อสร้างกระดานก่อนหน้าที่ถูกต้อง → แปลงเป็น SAN
            b_prev = labels_to_board(prev_labels)
            san = b_prev.san(mv)
            sans.append(san)
            prev_labels = now_labels  # อัปเดต state

    cap.release()
    return san_list_to_pgn(sans)


In [ ]:
rows = []
video_files = sorted(VIDEOS_DIR.glob("*.mp4"))
if not video_files:
    print(f"[warn] no videos found in: {VIDEOS_DIR}")

for v in tqdm(video_files, desc="Decoding videos"):
    row_id = v.stem
    pgn = decode_video_to_pgn(v)
    rows.append((row_id, pgn))
    print(f"{row_id} -> {pgn}")

with open(SUBMIT_CSV, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["row_id", "output"])
    w.writerows(rows)

print("✅ submission saved to:", SUBMIT_CSV.resolve())


Decoding videos:  20%|██        | 1/5 [04:02<16:08, 242.22s/it]

2_Move_rotate_student -> 


Decoding videos:  40%|████      | 2/5 [08:38<13:07, 262.40s/it]

2_move_student -> 


Decoding videos:  60%|██████    | 3/5 [18:29<13:45, 412.53s/it]

4_Move_studet -> 


Decoding videos:  80%|████████  | 4/5 [28:29<08:06, 486.37s/it]

6_Move_student -> 
